# Term Paper 
## Decision tree for asset price prediction

In [54]:
import pandas as pd

I utilize the dataset available online, https://www.kaggle.com/datasets/imranbukhari/comprehensive-btcusd-1m-data?select=BTCUSD_1m_Coinbase.csv, which represents the standard 'Open', 'Close', 'Low', 'High', 'Volume', 'Time' data for trading pair BTCUSD on 1 minute timeframe.

In [55]:
df = pd.read_csv("BTCUSD_1m_Coinbase.csv")

# Make sure 'Open time' is in datetime format
df['Open time'] = pd.to_datetime(df['Open time'])

# Set 'Open time' as index
df.set_index('Open time', inplace=True)

# Resample and aggregate
df = df.resample('60T').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).dropna()


## The Elder Impulse System

The Elder Impulse System was designed by Alexander Elder and featured in his book Come Into My Trading Room. According to Elder, “the system identifies inflection points where a trend speeds up or slows down.” The Impulse System is based on two indicators: a 13-day exponential moving average and the MACD-Histogram. The moving average identifies the trend, while the MACD-Histogram measures momentum. As a result, the Impulse System combines trend following and momentum to identify tradable impulses. This unique indicator combination is color coded into the price bars for easy reference.

Green Price Bar: (13-period EMA > previous 13-period EMA) and 
                 (MACD-Histogram > previous period's MACD-Histogram)

Red Price Bar: (13-period EMA < previous 13-period EMA) and 
               (MACD-Histogram < previous period's MACD-Histogram)

Price bars are colored blue when conditions for a Red Price Bar or 
Green Price Bar is not met. The MACD-Histogram is based on MACD(12,26,9). 

Green price bars show that the bulls are in control of trend and momentum as both the 13-day EMA and MACD-Histogram are rising. A red price bar indicates that the bears have taken control because the 13-day EMA and MACD-Histogram are falling. A blue price bar indicates mixed technical signals— neither buying nor selling pressure predominating.

In [56]:
# Step 1: Calculate indicators on Close
df['EMA_13'] = df['Close'].ewm(span=13, adjust=False).mean()
df['EMA_52'] = df['Close'].ewm(span=52, adjust=False).mean()
ema_12 = df['Close'].ewm(span=12, adjust=False).mean()
ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD_Line'] = ema_12 - ema_26
df['Signal_Line'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()
df['MACD_Hist'] = df['MACD_Line'] - df['Signal_Line']

# Step 2: Shift indicators to make them realistically usable at current bar
df['EMA_52_signal'] = df['EMA_52'].shift(1)
df['EMA_13_signal'] = df['EMA_13'].shift(1)
df['EMA_13_prev'] = df['EMA_13'].shift(2)
df['MACD_Hist_signal'] = df['MACD_Hist'].shift(1)
df['MACD_Hist_prev'] = df['MACD_Hist'].shift(2)

# Step 3: Apply bar coloring logic based only on available info
def color_bar(row):
    if pd.isna(row['EMA_13_signal']) or pd.isna(row['EMA_13_prev']):
        return None  # Skip early rows
    if row['EMA_13_signal'] > row['EMA_13_prev'] and row['MACD_Hist_signal'] > row['MACD_Hist_prev']:
        return 'Green'
    elif row['EMA_13_signal'] < row['EMA_13_prev'] and row['MACD_Hist_signal'] < row['MACD_Hist_prev']:
        return 'Red'
    else:
        return 'Blue'

df['Bar_Color'] = df.apply(color_bar, axis=1)

# Calculate daily range
df['Range'] = df['High'] - df['Low']

# ADR over last 10 bars, shifted to make it usable at the start of current bar
df['ADR_10'] = df['Range'].rolling(window=10).mean().shift(1)



In [57]:
# Rolling window size
window = 100

# Calculate rolling percentiles and median, shifted by 1 period
df['Upper 95'] = df['Close'].rolling(window).quantile(0.95).shift(1)
df['Lower 5'] = df['Close'].rolling(window).quantile(0.05).shift(1)
df['Median']   = df['Close'].rolling(window).median().shift(1)


In [58]:
import pandas as pd
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = df.tail(3000)

# Create subplots: 3 rows (Price + Indicators / MACD / Volume)
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    row_heights=[0.6, 0.2, 0.2],
    vertical_spacing=0.03,
    subplot_titles=('Price & Signals', 'MACD Histogram', 'Volume')
)

# Candles with color
for color, name in [('Green', 'green'), ('Red', 'red'), ('Blue', 'blue')]:
    subset = df[df['Bar_Color'] == color]
    fig.add_trace(go.Candlestick(
        x=subset.index,
        open=subset['Open'],
        high=subset['High'],
        low=subset['Low'],
        close=subset['Close'],
        increasing_line_color=name,
        decreasing_line_color=name,
        showlegend=False
    ), row=1, col=1)

# Add EMA and Rolling Window overlays
fig.add_trace(go.Scatter(
    x=df.index, y=df['EMA_13'],
    line=dict(color='orange', width=1),
    name='EMA 13'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df['EMA_52'],
    line=dict(color='purple', width=1),
    name='EMA 52'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df['Upper 95'],
    line=dict(color='white', width=1, dash='dot'),
    name='Upper 95%'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df['Lower 5'],
    line=dict(color='white', width=1, dash='dot'),
    name='Lower 5%'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df['Median'],
    line=dict(color='cyan', width=1, dash='dash'),
    name='Median'
), row=1, col=1)

# MACD Histogram
fig.add_trace(go.Bar(
    x=df.index,
    y=df['MACD_Hist'],
    marker_color=df['MACD_Hist'].apply(lambda x: 'green' if x >= 0 else 'red'),
    name='MACD Histogram'
), row=2, col=1)

# Volume
fig.add_trace(go.Bar(
    x=df.index,
    y=df['Volume'],
    marker_color='lightgray',
    name='Volume'
), row=3, col=1)

# Final layout tweaks
fig.update_layout(
    title='Trading Dashboard with Custom Coloring and Indicators',
    template='plotly_dark',
    xaxis_rangeslider_visible=False,
    height=900,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

# Export to HTML
plotly.offline.plot(fig, "CandleDashboard.html")


'temp-plot.html'

In [30]:
# Limit to last 1000 rows
df_last = df.tail(1000)

fig = go.Figure(go.Candlestick(
    x=df_last.index,
    open=df_last['Open'],
    high=df_last['High'],
    low=df_last['Low'],
    close=df_last['Close']
))

fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)


plotly.offline.plot(fig, filename="CandleStick.html")


'CandleStick.html'

In [61]:
RISK_REWARD_RATIO = 3
STOP_LOSS_ATR_MULTIPLIER = 1

# SL and TP levels
df['StopLoss'] = df['ADR_10'] * STOP_LOSS_ATR_MULTIPLIER
df['TakeProfit'] = df['StopLoss'] * RISK_REWARD_RATIO

df['SL_Long'] = df['Open'] - df['StopLoss']
df['TP_Long'] = df['Open'] + df['TakeProfit']
df['SL_Short'] = df['Open'] + df['StopLoss']
df['TP_Short'] = df['Open'] - df['TakeProfit'] 

def simulate_trade_classification(df):
    trade_signals = ['None'] * len(df)

    i = 0
    while i < len(df):
        row = df.iloc[i]

        if pd.isna(row['SL_Long']) or pd.isna(row['TP_Long']):
            i += 1
            continue

        # Simulate Long
        for j in range(i + 1, len(df)):
            future = df.iloc[j]
            if future['Low'] <= row['SL_Long']:
                break
            elif future['High'] >= row['TP_Long']:
                trade_signals[i] = 'Long'
                break

        # Simulate Short
        for j in range(i + 1, len(df)):
            future = df.iloc[j]
            if future['High'] >= row['SL_Short']:
                break
            elif future['Low'] <= row['TP_Short']:
                trade_signals[i] = 'Short'
                break

        i += 1

    df['Trade_Signal'] = trade_signals
    return df

df = simulate_trade_classification(df)


In [53]:
%reset -f